<img align="left" src = https://project.lsst.org/sites/default/files/Rubin-O-Logo_0.png width=250 style="padding: 10px"> 
<br>
<b> LSST Model Image </b><br>
Create a model image using outputs of LSST pipelines. <br> <br>

Contact author: Audrey Budlong <br>
Last verified to run: 16 February 2026 <br>

### Notebook Contents:
1. Imports
2. Setup
3. Define Butler
4. Select Data
5. Visualize Masks
6. Create Composite Model Image
7. Select Bright Sources
8. Visualize Results

### 1. Imports

In [ ]:
import crowdsource
import lsst.afw.display as afw_display
import lsst.geom as geom
import matplotlib.pyplot as plt
import numpy as np

from astropy import units as u
from astropy.visualization import simple_norm
from crowdsource.psf import SimplePSF
from crowdsource.crowdsource_base import fit_im
from lsst.daf.butler import Butler
from lsst.geom import SpherePoint, Angle, Point2D
from lsst.utils.plotting import (
    get_multiband_plot_colors,
    get_multiband_plot_symbols,
    get_multiband_plot_linestyles,
)
from scipy.ndimage import shift, zoom

### 2. Setup

In [ ]:
plt.style.use('seaborn-v0_8-colorblind')
prop_cycle = plt.rcParams['axes.prop_cycle']
colors = prop_cycle.by_key()['color']

In [ ]:
plt.rcParams['font.family'] = 'serif'

In [ ]:
filter_colors = get_multiband_plot_colors()
filter_symbols = get_multiband_plot_symbols()
filter_linestyles = get_multiband_plot_linestyles()

In [ ]:
afw_display.setDefaultBackend('firefly')
display = afw_display.Display(frame=1)

### 3. Define Butler

In [ ]:
collections = ["LSSTComCam/DP1"            ]
instrument="LSSTComCam"
skymap="lsst_cells_v1"
repo="/repo/dp1"

butler = Butler(repo, instrument=instrument, collections=collections, skymap=skymap)

In [ ]:
visit = 2024112600111
ccd = 8

tract = 10464
patch = 46

### 4. Select Data

In [ ]:
vi = butler.get("visit_image", dataId={"detector": ccd, "visit": visit})

In [ ]:
datareferences = butler.query_datasets("visit_image", where=f"detector={ccd} and visit={visit} and detector=8")
datareferences

In [ ]:
src_catalog = butler.get("source", dataId={"detector": ccd, "visit": visit})
src_catalog = src_catalog[src_catalog['detector']==ccd]
src_catalog

In [ ]:
src_catalog["coord_ra"][10]

In [ ]:
for column in src_catalog.columns:
    if "lux" in column:
        print(column)

In [ ]:
lsstSourceCat_flux = src_catalog["calibFlux"]
# lsstSourceCat_flux = src_catalog["psfFlux"]

In [ ]:
lsstSource_mag = (lsstSourceCat_flux*u.nJy).to(u.ABmag)
lsstSource_mag

In [ ]:
visit_summary = butler.get("visit_summary", dataId={"visit": visit, "detector": ccd})

In [ ]:
wcs = visit_summary.find(ccd).wcs

In [ ]:
sky_points = [
    SpherePoint(geom.Angle(ra, geom.degrees),
                geom.Angle(dec, geom.degrees))
    for ra, dec in zip(src_catalog['coord_ra'], src_catalog['coord_dec'])
]

pixel_points = wcs.skyToPixel(sky_points)

In [ ]:
pixel_points_x = [i.getX() for i in pixel_points]
pixel_points_y = [i.getY() for i in pixel_points]

In [ ]:
src_catalog["x"] = pixel_points_x
src_catalog["y"] = pixel_points_y
src_catalog["pixel points"] = pixel_points
src_catalog["magnitude"] = lsstSource_mag

In [ ]:
for column in src_catalog.columns:
    if "x" in column:
        print(column)

In [ ]:
vi_array = vi.image.array
norm = simple_norm(vi_array, stretch='asinh', percent=99.5)
im = plt.imshow(vi_array, cmap='gray', origin='lower', norm=norm)
for x, y in zip(src_catalog["x"], src_catalog["y"]):
    plt.scatter(x, y, marker='x', color='r')
for x, y in zip(pixel_points_x, pixel_points_y):
    plt.scatter(x, y, marker='x', color='r')
plt.xlim(300,500)
plt.ylim(3500,3800)

In [ ]:
psf_model = vi.getPsf()
def retrieve_psf(modelPsf, point):
    psf = modelPsf.computeImage(point)
    psf_array = np.array(psf.array)
    return psf_array

psf_list = []
for pixelPoint in pixel_points:
    lsst_psf = retrieve_psf(psf_model, pixelPoint)
    psf_list.append(lsst_psf)

In [ ]:
len(psf_list) == len(src_catalog)

### 5. Visualize Masks

In [ ]:
mask = vi.getMask()

# Get the bit value corresponding to the NOT_DEBLENDED plane
not_deblended_bit = mask.getPlaneBitMask("NOT_DEBLENDED")

# Mask array is integer-valued, one bit per plane
mask_array = mask.getArray()

# Boolean array: True where NOT_DEBLENDED is set
not_deblended = (mask_array & not_deblended_bit) != 0

In [ ]:
# --- Get visit image ---
vi = butler.get(
    "visit_image",
    dataId={"detector": ccd, "visit": visit}
)

image = vi.getImage().getArray()
mask = vi.getMask()
mask_array = mask.getArray()

h, w = mask_array.shape

# --- Define colors for mask planes (RGBA) ---
mask_colors = {
    "BAD":               [0.0, 1.0, 1.0, 1.0],  # cyan (stands out on gray)
    "SAT":               [1.0, 0.0, 1.0, 1.0],  # magenta
    "SAT_TEMPLATE":      [0.8, 0.0, 0.8, 1.0],
    "CR":                [0.0, 1.0, 0.5, 1.0],  # green-cyan
    "INTRP":             [0.0, 0.6, 1.0, 1.0],  # light blue
    "EDGE":              [1.0, 1.0, 0.0, 1.0],  # yellow
    "SENSOR_EDGE":       [1.0, 0.8, 0.0, 1.0],
    "DETECTED":          [0.0, 1.0, 0.0, 1.0],  # green
    "DETECTED_NEGATIVE": [0.0, 0.5, 0.0, 1.0],
    "NOT_DEBLENDED":     [1.0, 0.0, 0.0, 1.0],  # red
    "NO_DATA":           [0.0, 0.0, 0.0, 1.0],  # black
    "VIGNETTED":         [0.6, 0.3, 0.0, 1.0],  # brown
    "STREAK":            [1.0, 0.5, 0.0, 1.0],  # orange
    "SUSPECT":           [1.0, 0.0, 0.5, 1.0],  # pink
    "CLIPPED":           [0.5, 0.0, 1.0, 1.0],  # purple
    "CROSSTALK":         [0.0, 0.5, 0.5, 1.0],
    "INEXACT_PSF":       [0.3, 0.3, 1.0, 1.0],
    "INJECTED":          [0.0, 1.0, 0.5, 1.0],
    "INJECTED_TEMPLATE": [0.2, 0.8, 0.5, 1.0],
    "ITL_DIP":           [0.8, 0.2, 0.2, 1.0],
    "REJECTED":          [0.3, 0.0, 0.0, 1.0],
    "UNMASKEDNAN":       [1.0, 1.0, 1.0, 1.0],
}

# --- Build RGBA overlay (transparent background) ---
rgba = np.zeros((h, w, 4))

# Optional: define priority so important masks overwrite others
priority = [
    "NO_DATA",
    "EDGE",
    "SENSOR_EDGE",
    "BAD",
    "SAT",
    "CR",
    "STREAK",
    "NOT_DEBLENDED",
    "DETECTED",
]

for plane in priority:
    if plane not in mask_colors:
        continue
    if plane not in mask.getMaskPlaneDict():
        continue

    bit = mask.getPlaneBitMask(plane)
    plane_mask = (mask_array & bit) != 0
    rgba[plane_mask] = mask_colors[plane]

# --- Plot ---
plt.figure(figsize=(7, 7))

# Science image
plt.imshow(
    image,
    origin="lower",
    cmap="gray",
    vmin=np.percentile(image, 5),
    vmax=np.percentile(image, 95),
)

# Mask overlay
plt.imshow(rgba, origin="lower")

plt.title(f"Visit {visit}  Detector {ccd}")
plt.axis("off")
plt.tight_layout()
plt.show()

In [ ]:
# --- Get visit image ---
vi = butler.get(
    "visit_image",
    dataId={"detector": ccd, "visit": visit}
)

image = vi.getImage().getArray()
mask = vi.getMask()
mask_array = mask.getArray()

h, w = mask_array.shape

# --- Define colors for mask planes (RGBA) ---
mask_colors = {
    "BAD":               [0.0, 1.0, 1.0, 1.0],  # cyan
    "SAT":               [1.0, 0.0, 1.0, 1.0],
    "SAT_TEMPLATE":      [0.8, 0.0, 0.8, 1.0],
    "CR":                [0.0, 1.0, 0.5, 1.0],
    "INTRP":             [0.0, 0.6, 1.0, 1.0],
    "EDGE":              [1.0, 1.0, 0.0, 1.0],
    "SENSOR_EDGE":       [1.0, 0.8, 0.0, 1.0],
    "NOT_DEBLENDED":     [1.0, 0.0, 0.0, 1.0],
    "NO_DATA":           [0.0, 0.0, 0.0, 1.0],
    "VIGNETTED":         [0.6, 0.3, 0.0, 1.0],
    "STREAK":            [1.0, 0.5, 0.0, 1.0],
    "SUSPECT":           [1.0, 0.0, 0.5, 1.0],
    "CLIPPED":           [0.5, 0.0, 1.0, 1.0],
    "CROSSTALK":         [0.0, 0.5, 0.5, 1.0],
    "INEXACT_PSF":       [0.3, 0.3, 1.0, 1.0],
    "INJECTED":          [0.0, 1.0, 0.5, 1.0],
    "INJECTED_TEMPLATE": [0.2, 0.8, 0.5, 1.0],
    "ITL_DIP":           [0.8, 0.2, 0.2, 1.0],
    "REJECTED":          [0.3, 0.0, 0.0, 1.0],
    "UNMASKEDNAN":       [1.0, 1.0, 1.0, 1.0],
}

# --- Mask planes to EXCLUDE ---
exclude_planes = {
    "DETECTED",
    "DETECTED_NEGATIVE",
}

# --- Build RGBA overlay ---
rgba = np.zeros((h, w, 4))

# Priority order (detected intentionally omitted)
priority = [
    "NO_DATA",
    "EDGE",
    "SENSOR_EDGE",
    "BAD",
    "SAT",
    "CR",
    "STREAK",
    "NOT_DEBLENDED",
    "VIGNETTED",
    "SUSPECT",
    "CLIPPED",
]

for plane in priority:
    if plane in exclude_planes:
        continue
    if plane not in mask_colors:
        continue
    if plane not in mask.getMaskPlaneDict():
        continue

    bit = mask.getPlaneBitMask(plane)
    plane_mask = (mask_array & bit) != 0
    rgba[plane_mask] = mask_colors[plane]

# --- Plot ---
plt.figure(figsize=(7, 7))

plt.imshow(
    image,
    origin="lower",
    cmap="gray",
    vmin=np.percentile(image, 5),
    vmax=np.percentile(image, 95),
)

plt.imshow(rgba, origin="lower")

plt.title(f"Visit {visit}  Detector {ccd}")
plt.axis("off")
plt.tight_layout()
plt.show()


### 6. Create Composite Model Image

In [ ]:
def compositeImage(points, psfs, fluxes, pvi):
    H, W = pvi.image.array.shape
    composite = np.zeros((H, W), dtype=float)

    psf_h, psf_w = psfs[0].shape
    cy, cx = (psf_h-1) / 2, (psf_w-1) / 2  # keep as float for sub-pixel
    print(cy, cx)

    flux_cutouff = fluxes.max()/20

    for point, psf, flux in zip(points, psfs, fluxes):
        x = int(point.getX())
        y = int(point.getY())

        # Integer top-left corner in the image
        x0 = int(np.floor(x - cx))
        y0 = int(np.floor(y - cy))

        # Compute the slice in the image
        x1 = max(0, x0)
        y1 = max(0, y0)
        x2 = min(W, x0 + psf_w)
        y2 = min(H, y0 + psf_h)

        # # Corresponding PSF slice
        px1 = x1 - x0
        py1 = y1 - y0
        px2 = px1 + (x2 - x1)
        py2 = py1 + (y2 - y1)

        composite[y1:y2, x1:x2] += psf[py1:py2, px1:px2] * flux

    # Visualization
    norm = simple_norm(composite, stretch='asinh', percent=99.5)
    plt.figure()
    im = plt.imshow(composite, cmap='gray', origin='lower', norm=norm)
    plt.colorbar(im, label="PSF Calib Flux")
    plt.title("LSST Model Image")
    plt.show()

    return composite

In [ ]:
modelImageArray_subpixel = compositeImage(pixel_points, psf_list, lsstSourceCat_flux, vi)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (20,20))

vi_array = vi.image.array
vi_norm = simple_norm(vi_array, stretch='asinh', percent=99.5)

ax[0].imshow(vi_array, cmap='gray', origin='lower', norm=vi_norm)
for x, y in zip(src_catalog["x"], src_catalog["y"]):
    ax[0].scatter(x, y, color='r', marker='x', s=5)

ax[0].set_title("Visit Image")
ax[0].set_xlim(750,1250)
ax[0].set_ylim(0,500)

model_norm = simple_norm(modelImageArray_subpixel, stretch='asinh', percent=99.5)

ax[1].imshow(modelImageArray_subpixel, cmap='gray', origin='lower', norm=model_norm)
for x, y in zip(src_catalog["x"], src_catalog["y"]):
    ax[1].scatter(x, y, color='r', marker='x', s=5)
ax[1].set_title("LSST Model Image — Version 1")
ax[1].set_xlim(750,1250)
ax[1].set_ylim(0,500)

difference = vi_array - modelImageArray_subpixel
dfference_norm = simple_norm(difference, stretch='asinh', percent=99.5)

ax[2].imshow(difference, cmap='gray', origin='lower', norm=dfference_norm)
ax[2].set_title("Difference Image")
ax[2].set_xlim(750,1250)
ax[2].set_ylim(0,500)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (20,20))

vi_array = vi.image.array
vi_norm = simple_norm(vi_array, stretch='asinh', percent=99.5)

ax[0].imshow(vi_array, cmap='gray', origin='lower', norm=vi_norm)
ax[0].set_title("Visit Image")

model_norm = simple_norm(modelImageArray_subpixel, stretch='asinh', percent=99.5)

ax[1].imshow(modelImageArray_subpixel, cmap='gray', origin='lower', norm=model_norm)
ax[1].set_title("LSST Model Image — Version 1")

difference = vi_array - modelImageArray_subpixel
dfference_norm = simple_norm(difference, stretch='asinh', percent=99.5)

ax[2].imshow(difference, cmap='gray', origin='lower', norm=dfference_norm)
ax[2].set_title("Difference Image")

masked = np.ma.masked_where(~not_deblended, np.ones_like(not_deblended))

from matplotlib.colors import ListedColormap
cmap = ListedColormap([[1, 0, 0, 1]])  # solid red
cmap.set_bad(alpha=0)

ax[2].imshow(masked, origin="lower", cmap=cmap)

### 7. Select Bright Sources

In [ ]:
plt.hist(src_catalog["calibFlux"])

In [ ]:
src_catalog_flux_cap = src_catalog[src_catalog["calibFlux"]>=lsstSourceCat_flux.max()/200]
len(src_catalog_flux_cap)

In [ ]:
src_catalog_flux_cap_mid = src_catalog[(src_catalog["calibFlux"]>=lsstSourceCat_flux.max()/200) &
                                   (src_catalog["calibFlux"]<=lsstSourceCat_flux.max()/20)]
len(src_catalog_flux_cap_mid)

In [ ]:
src_catalog_flux_cap_high = src_catalog[src_catalog["calibFlux"]>=lsstSourceCat_flux.max()/2]
len(src_catalog_flux_cap_high)

### 8. Visualize Results

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (20,20))

difference = vi_array - modelImageArray_subpixel
difference_norm = simple_norm(difference, stretch='asinh', percent=99.5)

ax[0].imshow(vi_array, cmap='gray', origin='lower', norm=vi_norm)
ax[1].imshow(modelImageArray_subpixel, cmap='gray', origin='lower', norm=model_norm)
ax[2].imshow(difference, cmap='gray', origin='lower', norm=difference_norm)
for x, y in zip(src_catalog_flux_cap_high["x"], src_catalog_flux_cap_high["y"]):
    ax[2].scatter(x, y, facecolor='none', edgecolor='r', marker='o', s=40)

In [ ]:
fig, ax = plt.subplots(1, 3, figsize = (20,20))

difference = vi_array - modelImageArray_subpixel
difference_norm = simple_norm(difference, stretch='asinh', percent=99.5)

ax[0].imshow(vi_array, cmap='gray', origin='lower', norm=vi_norm)
ax[1].imshow(modelImageArray_subpixel, cmap='gray', origin='lower', norm=model_norm)
ax[2].imshow(difference, cmap='gray', origin='lower', norm=difference_norm)